# Building Recommendation Engine



1.	From this URL (https://www.zomato.com/bangalore/great-food-no-bull), scrape the following information and store them in a data frame
a.	Restaurant Name
b.	URL to the restaurant
2.	Loop through each restaurant and do the following
a.	Check if you are able to scrape sample reviews. Because certain URLs will take you to restaurant’s branches page. You can skip these kind of restaurants 
b.	If you are able to scrape sample reviews, make sure you click the load more button continuously in a loop until all reviews are loaded
c.	From each review extract the following
i.	Reviewer name
ii.	Reviewer ID
iii.	Ratings
3.	Repeat the above process for atleast 50 restaurants. Store all the results in the data frame with the following structure
Restaurant Name	Restaurant ID	User Name	User ID	Rating

4.	Perform exploratory data analysis using the above data
5.	User the above data, build a recommendation engine using User Based Collaborative Filtering. Create a function which takes User ID as input and return 3 restaurants that you would recommend to the user. Make sure you show a sample input and output. 

Note for Submission
•	Submit zipped version of your Jupyter Notebook in HTML format. 
•	Make sure that both zip file and HTML file contains your name
•	If few restaurants throw error while scraping, you can skip them using try and except blocks
•	Make sure that you have print statements wherever necessary


#### importing libraries

In [1]:
import pandas as pd
import bs4
import selenium
import requests
import urllib
import csv
from sklearn.metrics.pairwise import cosine_similarity

#### reading the zomato url

In [2]:
from selenium import webdriver
browser = webdriver.Chrome('chromedriver.exe')
url1 = 'https://www.zomato.com/bangalore/great-food-no-bull'
browser.get(url1)

In [3]:
urls = browser.find_elements_by_css_selector('div.relative.top-res-box.entity-ads-snippet-track')

#### 	From this URL (https://www.zomato.com/bangalore/great-food-no-bull), scrape the following information and store them in a data frame
- Restaurant Name
- URL to the restaurant


In [4]:
urls_df = pd.DataFrame(columns=['Restaurant_Name', 'URL'])
for elem in urls:
    url = elem.find_element_by_css_selector('a')
    url = url.get_attribute('href')
    rest_name = elem.find_element_by_css_selector('div.res_title')
    rest_name = rest_name.get_attribute('innerHTML')
    curr_row = {
        'URL' : url, 
        'Restaurant_Name' : rest_name
    }
    
    urls_df = urls_df.append(curr_row,ignore_index = True)
    urls_df['Restaurant_Name'] = urls_df['Restaurant_Name'].str.replace('\n','')
    row = [url,rest_name.strip()]
    with open('rest.csv', 'a', newline = '') as csvFile: ### writing the dataframe to a csv file
        writer = csv.writer(csvFile)
        writer.writerow(row)

In [5]:
urls_df

,Restaurant_Name,URL
0,The Pallet,https://www.zomato.com/bangalore/the-pallet-wh...
1,1131 Bar + Kitchen,https://www.zomato.com/bangalore/1131-bar-kitc...
2,Simon Says Brew Works,https://www.zomato.com/bangalore/simon-says-br...
3,The Fatty Bao - Asian...,https://www.zomato.com/bangalore/the-fatty-bao...
4,Burma Burma,https://www.zomato.com/bangalore/burma-burma-i...
5,The Reservoire,https://www.zomato.com/bangalore/the-reservoir...
6,Market Table,https://www.zomato.com/bangalore/market-table-...
7,XOOX Brewmill,https://www.zomato.com/bangalore/xoox-brewmill...
8,Baar Union,https://www.zomato.com/bangalore/baar-union-hs...
9,Big Pitcher,https://www.zomato.com/bangalore/big-pitcher-a...


####  Loop through each restaurant(atleast 50) and do the following
- scrape sample reviews, make sure you click the load more button continuously in a loop until all reviews are loaded
- From each review extract the following
    - Reviewer name
    - Reviewer ID
    - Ratings


In [ ]:
reviews_df = pd.DataFrame(columns=['Restaurant_name','User_ID','User_Name','Rating']) #,'Reviews'
for a,elem1 in urls_df.iterrows():
    try: ## to catch the exceptions that we get while scraping data from restaurants(to skp these kind of restaurantsS)
        browser.get(elem1['URL'])
        rest_name = elem1['Restaurant_Name']
        while(True): ### load more
            try:
                more_sel = 'div.load-more'
                load_more_button = browser.find_element_by_css_selector(more_sel)
                load_more_button.click()
            except Exception:
                break;
        reviews = browser.find_elements_by_css_selector('div.ui.segment.clearfix.brtop')
        for elem in reviews:
            name = elem.find_element_by_css_selector('a[data-entity_id]')
            uid = name.get_attribute('data-entity_id')
            name = name.get_attribute('innerHTML')#.splitlines()[1]
            rating =  elem.find_element_by_css_selector('div[aria-label]')
            rating = rating.get_attribute('aria-label')
            curr_row = {
                'Restaurant_name' : rest_name,
                'User_ID' : uid,
                'User_Name': name,
                'Rating' : rating,
                #'Reviews' : rev
            }

            reviews_df = reviews_df.append(curr_row,ignore_index = True)
            reviews_df['Rating'] = reviews_df['Rating'].str.replace('[^0-9.]','')
            reviews_df['User_Name'] = reviews_df['User_Name'].str.replace('[^a-zA-Z]','')
            #reviews_df['Reviews'] = reviews_df['Reviews'].str.replace('[^a-zA-Z0-9 ]','')
            #reviews_df['Reviews'] = reviews_df['Reviews'].str.replace('RATED','')
            row = [rest_name.strip(), uid,name.strip(),rating]
            with open('review.csv', 'a', newline='') as csvFile: ### writing the dataframe to a csv file
                writer = csv.writer(csvFile)
                writer.writerow(row)
    except Exception:
                continue
    if(str(a)=='70'): ### restricting the data to 70 rows so that we get details of atleast 50 restaurants
        break

#### writing the data from dataframe to a csv file

In [ ]:
reviews = pd.read_csv('review.csv', encoding = "latin")
reviews.head()

#### pivot table for the reviews DF

In [ ]:
user_item_matrix = reviews.pivot_table(index='User_ID',
                   columns = 'Restaurant_Name',
                   values='Rating')
print(user_item_matrix.shape)
user_item_matrix.head()

#### Perform exploratory data analysis using the above data

In [ ]:
# 1. Avg no. of ratings per user

user_item_matrix.count(axis=1).mean() ## per user avg

In [ ]:
# 2. User with most no. of ratings

user_item_matrix.count(axis=1).sort_values(ascending = False).head(1)

In [ ]:
# 3. Avg rating per restaurant

user_item_matrix.count().mean()

In [ ]:
# 4. Restaurant with most no. of ratings

user_item_matrix.count().sort_values(ascending = False).head(1)

In [ ]:
## cosine_similarity does not accept missing values
 
ui_matrix = user_item_matrix.apply(lambda v: v - v.mean(), axis = 1).fillna(0)

In [ ]:

sim_matrix = pd.DataFrame(cosine_similarity(ui_matrix),
            index = ui_matrix.index,
            columns=ui_matrix.index)

In [ ]:
sim_matrix

#### 5.	User the above data, build a recommendation engine using User Based Collaborative Filtering. Create a function which takes User ID as input and return 3 restaurants that you would recommend to the user.

In [ ]:
def ubcf(userid): 
    preds = pd.DataFrame(columns=['rname', 'predicted_rating'])
    neighs_user = sim_matrix[userid].drop(userid).sort_values(ascending = False).head(7).index
    for rname, rating in user_item_matrix.loc[userid].items(): 
        if pd.isnull(rating):
            rname_ratings = user_item_matrix.loc[neighs_user, rname]
            avg_rname_rating = rname_ratings.mean() # predicting the rating for the restaurants based on the avg rating given by neighbours
            preds = preds.append({'rname' : rname,
                                 'predicted_rating': avg_rname_rating},
                                 ignore_index = True)
            rest_recom = preds.sort_values('predicted_rating', ascending=False).head(3)['rname']
    return rest_recom.values.tolist()

#### Sample input and output. 

In [ ]:
ubcf(19800)